In [276]:
scrape_forecast_weather()

ValueError: cannot reindex from a duplicate axis

In [124]:
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup as bs
import datetime
import calendar
from dateutil.relativedelta import *
import smtplib
from getpass import getpass
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import os

In [274]:
def scrape_forecast_weather():
# Function returns a dataframe with 10-day ahead 24-h weather forecast, starting day after the current day
# Full URL: https://www.wunderground.com/hourly/ca/toronto/date/2018-9-25?cm_ven=localwx_hour

    days_ahead = 1 
    forecast_out = pd.DataFrame() # Will contain 'day_out' for all days in days_ahead 
    
    while days_ahead < 10:    
    
        # Relativedelta calculates the calendar day of n days_ahead
        now = datetime.datetime.now()
        day = (now + relativedelta(days =+ days_ahead)).day
        month = (now + relativedelta(days =+ days_ahead)).month
        year = (now + relativedelta(days =+ days_ahead)).year
        weekday = (now + relativedelta(days =+ days_ahead)).weekday()

        # Requests content from a url for each calendar day in days_ahead
        url = 'https://www.wunderground.com/hourly/ca/toronto/date/'\
                +str(year)+'-'+str(month)+'-'+str(day)+'?cm_ven=localwx_hour' 
        
        request = requests.get(url)
        content = request.content
        soup = bs(content,'html.parser') 
        
        # Finds table rows on page, parses and appends to a list:
        table_in = [each for each in soup.find_all('tr') if 'header' not in str(each)]
        table_out = []

        for row_in in table_in:
            row_out = parse_row(row_in)
            table_out.append(row_out)

        # Table_out modified (adds header, converts to dataframe, adds date columns) --> day_out:
        header = list(soup.find_all('tr')[0].stripped_strings)
        day_out = pd.DataFrame(table_out, columns=header)
        day_out.insert(loc = 0, column = "Year", value = (year))
        day_out.insert(loc = 1, column = "Month", value = (month))
        day_out.insert(loc = 2, column = "Day", value = (day))
        day_out.insert(loc = 3, column = "Weekday", value = (weekday))
        
        forecast_out = forecast_out.append(day_out)        
        days_ahead += 1

    # Removes non-digit characters and coverts to Celcius:
    
    forecast_out = (forecast_out
            #           .reset_index()
                        .assign(
                                temp=forecast_out['Temp.'].apply(lambda x: int(re.sub("\D","",str(x)))),
                                dew_point=forecast_out['Dew Point'].apply(lambda x: int(re.sub("\D","",str(x)))),
                                r_h=forecast_out['Humidity'].apply(lambda x: int(re.sub("\D","",str(x)))),
                                hour=forecast_out['Time'].apply(lambda x: int(re.sub("\D","",str(x))))
                               )
                   )
    forecast_out = (forecast_out                   
                        .assign(
                                temp_c=forecast_out['temp'].apply(lambda x: round((int(x)-32)*(5/9),1)),
                                dew_point_c=forecast_out['dew_point'].apply(lambda x: round((int(x)-32)*(5/9),1)),
                                hour=forecast_out['hour'].apply(lambda x: int(re.sub("\d\d$","",str(x))))
                               )
                                [['hour',
                                'temp_c',
                                'dew_point_c',
                                'r_h']]
                   )
    
    # for i in [year, month, day, weekday]:
       # forecast_out.insert(loc = 0, column = str(i), value = i )
                                    
    print(forecast_out.count())
    return forecast_out.tail()

def parse_row(row):
# Returns a row as a list of cells
    cells_in = [text for text in row.find_all('td')]
    cells_out = []
    
    # Joins multi-string cells into single cells
    for cell in cells_in:
        strings = cell.stripped_strings
        cell_contents = []
        
        for string in strings:
            cell_contents.append(string)
            
        cell_out = "".join(cell_contents)
        cells_out.append(cell_out)
        
    return cells_out


In [ ]:
#forecast_out.to_csv('~/Desktop/weather_forecast.csv')

In [272]:
def scrape_forecast_weather():
# Function returns a dataframe with 10-day ahead 24-h weather forecast, starting day after the current day
# Full URL: https://www.wunderground.com/hourly/ca/toronto/date/2018-9-25?cm_ven=localwx_hour

    days_ahead = 0 # Starting with current day may mess up index of day_out (due to <24 rows). Same-day functionality not required at this time.
    forecast_out = pd.DataFrame() # Will contain 'day_out' for all days in days_ahead 
    
    while days_ahead < 10:    
    
        # Relativedelta calculates the calendar day of n days_ahead:
        now = datetime.datetime.now()
        day = (now + relativedelta(days =+ days_ahead)).day
        month = (now + relativedelta(days =+ days_ahead)).month
        year = (now + relativedelta(days =+ days_ahead)).year
        weekday = (now + relativedelta(days =+ days_ahead)).weekday()

        # Requests gets content from a url for each calendar day in days_ahead:
        url = 'https://www.wunderground.com/hourly/ca/toronto/date/'+str(year)+'-'+str(month)+'-'+str(day)+'?cm_ven=localwx_hour'       
        request = requests.get(url)
        content = request.content
        soup = bs(content,'html.parser') 
        
        # Finds table rows on page, parses and appends to a list:
        table_in = [each for each in soup.find_all('tr') if 'header' not in str(each)]
        table_out = []

        for row_in in table_in:
            row_out = parse_row(row_in)
            table_out.append(row_out)

        # Table_out modified (adds header, converts to dataframe, adds date columns) --> day_out:
        header = list(soup.find_all('tr')[0].stripped_strings)
        day_out = pd.DataFrame(table_out, columns=header)
        day_out.insert(loc = 0, column = "Year", value = (year))
        day_out.insert(loc = 1, column = "Month", value = (month))
        day_out.insert(loc = 2, column = "Day", value = (day))
        day_out.insert(loc = 3, column = "Weekday", value = (weekday))
        #day_out.index.names = ['Hour']

        # Day_out appended to forecast_out
        forecast_out = forecast_out.append(day_out)   
        days_ahead += 1
         

    # Removes non-digit characters:
    forecast_out["Temp."] = forecast_out["Temp."].apply(lambda x: int(re.sub("\D","",str(x))))
    forecast_out["Dew Point"] = forecast_out["Dew Point"].apply(lambda x: int(re.sub("\D","",str(x))))
    forecast_out["Humidity"] = forecast_out["Humidity"].apply(lambda x: int(re.sub("\D","",str(x))))
    forecast_out["Time"] = forecast_out["Time"].apply(lambda x: int(re.sub("\D","",str(x))))
    forecast_out["Time"] = forecast_out["Time"].apply(lambda x: int(re.sub("\d\d$","",str(x))))
    
    # Converts F to C
    forecast_out["Temp."] = forecast_out["Temp."].apply(lambda x: round((int(x)-32)*(5/9),1))
    forecast_out["Dew Point"] = forecast_out["Dew Point"].apply(lambda x: round((int(x)-32)*(5/9),1))
    
    # Standardizes with Excel weekday formula (1-7)
    forecast_out["Weekday"] = forecast_out["Weekday"].apply(lambda x:x+1)
    
    # uses old index as 'Hour' and resets index
    # forecast_out['Hour'] = forecast_out.index
    forecast_out.reset_index(inplace=True)
   
    # Drops, renames and re-arranges columns:    
    forecast_out.rename(columns={
        "Humidity": "Relative Humidity", 
        "Temp.": "Temp C"
    },inplace=True)
    
    forecast_out.drop([
        'Conditions',
        'index',
        'Feels Like',
        'Pressure',
        'Precip',
        'Amount',
        'Wind',
        'Cloud Cover'
    ],axis=1,inplace = True)
    
    # Do we need to drop (above) if we're doing this????
    forecast_out = forecast_out[[
        'Year',
        'Month',
        'Weekday',
        'Day',
        'Time',
        'Temp C',
        'Dew Point',
        'Relative Humidity'
    ]]
    
    #forecast_out.to_csv('~/Desktop/weather_forecast.csv')
    
    print(forecast_out.count())
    return forecast_out.tail()

def parse_row(row_in):
# Function returns a list containing data as strings, for a single 'row_in'

    cells_in = [text for text in row_in.find_all('td')]
    row_out = []
    
    # Some cells (divs?) contain multiple strings, these are joined so each cell is a single list item in row_out:
    for cell in cells_in:
        strings = cell.stripped_strings
        cell_contents = []
        
        for string in strings:
            cell_contents.append(string)
            
        cell_out = "".join(cell_contents)
        row_out.append(cell_out)
        
    return row_out
 

In [222]:
        now = datetime.datetime.now()
        day = (now + relativedelta(days =+ 1)).day
        month = (now + relativedelta(days =+ 1)).month
        year = (now + relativedelta(days =+ 1)).year
        weekday = (now + relativedelta(days =+ 1)).weekday()

        # Requests content from a url for each calendar day in days_ahead
        url = 'https://www.wunderground.com/hourly/ca/toronto/date/'\
                +str(year)+'-'+str(month)+'-'+str(day)+'?cm_ven=localwx_hour' 
        
        request = requests.get(url)
        content = request.content
        soup = bs(content,'html.parser')